# Limpieza de los datos
En esta sección vamos clasificar cada columna del set de datos por un tipo específico e.g., categóricos, numéricos, cadenas, etc. y decidir que hacer con los posibles atributos que no respeten dicha clasificación.

In [1]:
if 'limpieza_runned' in locals():
    raise KeyboardInterrupt('La limpieza ya corrió en este Kernel')
limpieza_runned = True

In [2]:
import pandas as pd
import numpy as np
import os

In [3]:
pd.set_option('display.max_columns', 25)

In [4]:
if '__file__' in locals():
    current_folder = os.path.dirname(os.path.abspath(__file__))
else:
    current_folder = os.getcwd()

In [5]:
dataset = 'events_up_to_01062018.csv'
df = pd.read_csv(os.path.join(current_folder, 'fiuba-trocafone-tp2-final-set', dataset), low_memory=False)

## Clasificación de los datos
Debido a la descripción proporcionada en el enunciado del trabajo práctico espero que los siguientes atributos sean __categoricos__:
- [event](#event)
- [person](#person)
- [url](#url)
- [sku](#sku)
- [model](#model)
- [condition](#condition)
- [storage](#storage)
- [color](#color)
- [skus](#skus)
- [staticpage](#staticpage)
- [campaign_source](#campaign_source)
- [search_engine](#search_engine)
- [channel](#channel)
- [new_vs_returning](#new_vs_returning)
- [city](#city)
- [region](#region)
- [country](#country)
- [device_type](#device_type)
- [operating_system_version](#operating_system_version)
- [browser_version](#browser_version)

Para comprobarlo transformo los datos en categóricos y observo qué categorías se generan.

Por otra parte espero que los siguiente atributos no lo sean:
- [screen_resolution](#screen_resolution)
- [timestamp](#timestamp)

In [6]:
atributos_categoricos = ['event', 'person', 'url', 'sku', 'model', 'condition', 'storage', 'color', 'staticpage',
                        'campaign_source', 'search_engine', 'channel', 'new_vs_returning', 'city', 'region', 'country',
                        'device_type', 'operating_system_version', 'browser_version']
for atributo in atributos_categoricos:
    df[atributo] = df[atributo].astype('category')

Ahora podemos revisar las categorías de cada atributo:

<h4 id="event">event</h4>

In [7]:
df['event'].cat.categories

Index(['ad campaign hit', 'brand listing', 'checkout', 'conversion',
       'generic listing', 'lead', 'search engine hit', 'searched products',
       'staticpage', 'viewed product', 'visited site'],
      dtype='object')

In [8]:
df[pd.isnull(df['event'])].size

0

En este caso vemos que los eventos coinciden con los descritos en el enunciado y no tenemos ningún nulo.

<h4 id="person">person</h4>

In [9]:
df['person'].cat.categories

Index(['0008ed71', '00091926', '00091a7a', '000ba417', '000c79fe', '000e4d9e',
       '000e619d', '001001be', '0010e89a', '0016c4b5',
       ...
       'ffee0f18', 'ffef83e6', 'fff1659c', 'fff1b11a', 'fff1caee', 'fff2bdde',
       'fff54d61', 'fff72025', 'fff78145', 'fffd1246'],
      dtype='object', length=38829)

Notamos que las categorias de los usuarios se parecen mucho a números en hexadecimal. Podriamos pasarlo a binario o a decimal pero no ganariamos nada dado que no resulta util calcular promedios, varianzas, etc. sobre un identificador. Por otro lado puede resultar interesante confirmar si los datos de esta columna se corresponden a números en hexadecimal simplemente para confirmar la integridad de los datos e identificar anomalías.

In [10]:
person_in_decimal = df['person'].apply(lambda x: int(x, 16))
pd.to_numeric(person_in_decimal);

Vemos que no hubo problemas en la conversion.

In [11]:
df[pd.isnull(df['person'])].size

0

A su vez vemos que no hay ningún valor nulo.

<h4 id="url">url</h4>

In [12]:
df['url'].cat.categories

Index(['/', '/comprar/apple/iphone-6s-plus', '/comprar/apple/iphone-7',
       '/comprar/apple/iphone-7-plus', '/comprar/asus/asus-live',
       '/comprar/asus/zenfone-2', '/comprar/asus/zenfone-2-deluxe',
       '/comprar/asus/zenfone-2-laser', '/comprar/asus/zenfone-3-max-16gb',
       '/comprar/asus/zenfone-3-max-32gb',
       ...
       '/vender/lg/optimus-l7-ii',
       '/vender/motorola/moto-e-2a-geracao-4g-dual',
       '/vender/motorola/moto-g5-plus', '/vender/motorola/moto-g5s-plus',
       '/vender/motorola/moto-maxx-2a-geracao',
       '/vender/motorola/moto-x-play-4g-dual', '/vender/motorola/moto-z2-play',
       '/vender/motorola/motorola-moto-g-2a-geracao-4g-dual',
       '/vender/samsung/galaxy-s5', '/vender/samsung/galaxy-s8'],
      dtype='object', length=248)

In [13]:
df[pd.isnull(df['url'])].shape

(2150550, 23)

En este caso sí hay varios nulos.

<h4 id="sku">sku</h4>

In [14]:
df['sku'].cat.categories

Float64Index([   71.0,    74.0,    80.0,    81.0,    87.0,    88.0,    94.0,
                 95.0,   101.0,   102.0,
              ...
              17170.0, 17183.0, 17184.0, 17185.0, 17198.0, 17199.0, 17200.0,
              17483.0, 17484.0, 17485.0],
             dtype='float64', length=2328)

En este caso vemos que hay varias categorias numéricas (aunque por ahora no los convertimos) y una 'undefined'. Primero vamos a ver que hay dentro de 'undefined' y luego trataremos de convertirlas a datos numéricos.

In [15]:
undefined_skus = df[df['sku'] == 'undefined']

Se ve que son solo dos y que todos los atributos son valores nulos exceptuando el de persons, así que sospechamos que son eventos que no aportan informción realmente. Para confirmarlo podemos buscar otros eventos realizados por los mismos usuarios y ver si hay otro en un timestamp cercano.

In [16]:
df[df['person'].isin(undefined_skus['person'])]

,timestamp,event,person,url,sku,model,condition,storage,color,skus,search_term,staticpage,campaign_source,search_engine,channel,new_vs_returning,city,region,country,device_type,screen_resolution,operating_system_version,browser_version


Se ve que en ambos casos existe otro evento de tipo 'visited site' a pocos segundos del primer evento, por lo cual podemos descartarlos sabiendo que existe otro con mas información que representa la visita del usuario.

In [17]:
df.drop(index=undefined_skus.index, inplace=True)

Ahora podemos transformar el 'sku' a un tipo numerico.

In [18]:
df['sku'] = pd.to_numeric(df['sku'])

Y ahora ya nos aseguramos que todos son del mismo tipo podemos volver a convertirlos en categorias.

In [19]:
df['sku'] = df['sku'].astype('category')

<h4 id="model">model</h4>

In [20]:
df['model'].cat.categories

Index(['Asus Live', 'Asus Zenfone 2', 'Asus Zenfone 2 Deluxe',
       'Asus Zenfone 2 Laser', 'Asus Zenfone 2 Laser 6"',
       'Asus Zenfone 3 Max  32 GB', 'Asus Zenfone 3 Max 16 GB',
       'Asus Zenfone 5', 'Asus Zenfone 6', 'Asus Zenfone Go',
       ...
       'iPhone 6', 'iPhone 6 Plus', 'iPhone 6S', 'iPhone 6S Plus', 'iPhone 7',
       'iPhone 7 Plus', 'iPhone 8', 'iPhone 8 Plus', 'iPhone SE', 'iPhone X'],
      dtype='object', length=208)

<h4 id="condition">condition</h4>

In [21]:
df['condition'].cat.categories

Index(['Bom', 'Bom - Sem Touch ID', 'Excelente', 'Muito Bom', 'Novo'], dtype='object')

Acá notamos que los valores de este atributo están en portugués, por lo que aprovechamos para traducirlos a español y así trabajar más comodamente.

In [22]:
condition_translations = {
    'Bom': 'Bueno',
    'Bom - Sem Touch ID': 'Bueno - Sin Touch ID',
    'Excelente': 'Excelente',
    'Muito Bom': 'Muy bueno',
    'Novo': 'Nuevo'
}

df['condition'].cat.rename_categories(condition_translations, inplace=True)

In [23]:
df['condition'].cat.categories

Index(['Bueno', 'Bueno - Sin Touch ID', 'Excelente', 'Muy bueno', 'Nuevo'], dtype='object')

<h4 id="storage">storage</h4>

In [24]:
df['storage'].cat.categories

Index(['128GB', '16GB', '256GB', '32GB', '4GB', '512MB', '64GB', '8GB'], dtype='object')

Puede resultar buena idea pasar este atributo a valores numéricos dado que la capacidad de almacenamiento es un valor numérico, pero por otro lado sabemos que la capacidad de los smartphones toma solo ciertos valores determinados por lo que resulta conveniente dejarlo como categorías.
Sin embargo sí resultaría muy útil medir todos los datos con la misma unidad y dado que observamos que la mayoría de los datos están en GB decidimos utilizar esta unidad.

In [25]:
storage_translations = {}

for almacenamiento in df['storage'].cat.categories:
    capacidad = float(almacenamiento[:-2])
    storage_translations[almacenamiento] = capacidad
storage_translations['512MB'] = 0.5

df['storage'].cat.rename_categories(storage_translations, inplace=True)

In [26]:
df['storage'].cat.categories

Float64Index([128.0, 16.0, 256.0, 32.0, 4.0, 0.5, 64.0, 8.0], dtype='float64')

A su vez para dejar en claro la unidad en la que estamos trabajando renombraremos la columna a 'storage_gb'.

In [27]:
df.rename(columns={'storage': 'storage_gb'}, inplace=True);

Y también cambiamos el nombre en nuestra lista de atributos categóricos.

In [28]:
atributos_categoricos[6] = 'storage_gb'

<h4 id="color">color</h4>

In [29]:
df['color'].cat.categories

Index(['Amarelo', 'Ametista', 'Azul', 'Azul Escuro', 'Azul Safira',
       'Azul Topázio', 'Bambu', 'Black Piano', 'Branco', 'Branco Azul',
       'Branco Azul Navy', 'Branco Bambu', 'Branco Cabernet', 'Branco Dourado',
       'Branco Framboesa', 'Branco Pink', 'Branco Verde', 'Branco Vermelho',
       'Cabernet', 'Cinza', 'Cinza espacial', 'Cobre', 'Coral', 'Couro Marrom',
       'Couro Navy', 'Couro Vinho', 'Couro Vintage', 'Cromo', 'Dourado',
       'Framboesa', 'Indigo', 'Iuna', 'Olympic Edition', 'Ouro', 'Ouro Rosa',
       'Platinum', 'Prata', 'Prateado', 'Preto', 'Preto Asfalto', 'Preto Azul',
       'Preto Azul Navy', 'Preto Bambu', 'Preto Branco', 'Preto Brilhante',
       'Preto Cabernet', 'Preto Matte', 'Preto Pink', 'Preto Tabaco',
       'Preto Verde', 'Preto Vermelho', 'Rosa', 'Rose', 'Rouge', 'Roxo',
       'Silver', 'Titânio', 'Turquesa', 'Verde', 'Verde Petroleo',
       'Verde Água', 'Vermelho', 'Ônix'],
      dtype='object')

Al igual que con el almacenamiento podemos traducir las categorias a español.

En algunos casos se encontraron colores un poco 'extravagantes' o sin un significado concreto, por lo que se procedió a buscar más información sobre el mismo a partir del modelo de smartphone al que corresponden. Un ejemplo de ello es el color 'Iuna':

In [30]:
df[(df['color'] == 'Iuna')]

,timestamp,event,person,url,sku,model,condition,storage_gb,color,skus,search_term,staticpage,campaign_source,search_engine,channel,new_vs_returning,city,region,country,device_type,screen_resolution,operating_system_version,browser_version
954,2018-05-29 02:57:26,viewed product,c0d67259,NaN,3095.0,Motorola Moto X2,Bueno,32.0,Iuna,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5763,2018-05-19 07:36:30,viewed product,429c8732,NaN,3095.0,Motorola Moto X2,Bueno,32.0,Iuna,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11297,2018-05-30 05:19:48,viewed product,8bb51947,NaN,3095.0,Motorola Moto X2,Bueno,32.0,Iuna,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11361,2018-05-30 05:20:01,viewed product,8bb51947,NaN,3095.0,Motorola Moto X2,Bueno,32.0,Iuna,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17227,2018-05-15 17:08:48,viewed product,da1cc049,NaN,3095.0,Motorola Moto X2,Bueno,32.0,Iuna,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18826,2018-05-15 13:36:32,viewed product,5ccaa0a7,NaN,3095.0,Motorola Moto X2,Bueno,32.0,Iuna,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20101,2018-05-15 01:01:35,viewed product,94e2a3c9,NaN,3095.0,Motorola Moto X2,Bueno,32.0,Iuna,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20370,2018-05-15 12:39:54,viewed product,8ceb42a4,NaN,3095.0,Motorola Moto X2,Bueno,32.0,Iuna,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21062,2018-05-15 02:20:18,viewed product,ab9fd1f7,NaN,3095.0,Motorola Moto X2,Bueno,32.0,Iuna,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21214,2018-05-15 23:45:01,viewed product,ab9fd1f7,NaN,3095.0,Motorola Moto X2,Bueno,32.0,Iuna,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
df[(df['color'] == 'Iuna') & (df['model']!= 'Motorola Moto X2')].size

0

Al buscar el modelo 'Motorola Moto X2' (que es el único que aparece en el set de datos con el color 'Iuna') en www.trocafone.com, se encontró que dicho color pertenecía a una smartphone con un dibujo de madera en la carcaza (https://www.trocafone.com/comprar/motorola/moto-x-2a-geracao#242-motorola-moto-x2-32gb-iuna-bom-15). Por ello se procedió a tracucir este color como 'Madera'.

Algo similar ocurrió con otros colores, como por ejemplo el 'Branco Azul'. En este caso vemos que la traducción literal sería  'Blanco Azul'. Esto pordría significar celeste, o también un smartphone con dos colores. Para comprobarlo buscamos los modelos con este color:

In [32]:
df[df['color'] == 'Branco Azul']['model'].unique()

[Motorola Moto G3 4G, Motorola Moto G3 HDTV]
Categories (2, object): [Motorola Moto G3 4G, Motorola Moto G3 HDTV]

Luego al buscar estos modelos notamos que son blancos en la parte delantera y azules en la trasera. Para reflejar esto de manera más sencilla se utilizó como traducción 'Blanco,Azul', de manera tal que cuando se encuentre un smartphone con una el formato 'color1,color2' se entienda que el color1 es el de la parte delantera y color2 el de la trasera. Si se encuentra el formato 'color', se entenderá que el celuar es de un solo color.

In [33]:
color_translations = {
    'Amarelo': 'Amarillo',
    'Ametista': 'Amatista',
    'Azul': 'Azul',
    'Azul Escuro': 'Azul Oscuro',
    'Azul Safira': 'Azul Zafiro',
    'Azul Topázio': 'Azul Topacio',
    'Bambu': 'Bambu',
    'Black Piano': 'Negro Piano',
    'Branco': 'Blanco',
    'Branco Azul': 'Blanco,Azul',
    'Branco Azul Navy': 'Blanco,Azul Navy',
    'Branco Bambu': 'Blanco,Bambu',
    'Branco Cabernet': 'Blanco,Cabernet',
    'Branco Dourado': 'Blanco,Dorado',
    'Branco Framboesa': 'Blanco,Franbuesa',
    'Branco Pink': 'Blanco,Rosa',
    'Branco Verde': 'Blanco,Verde',
    'Branco Vermelho': 'Blanco,Rojo',
    'Cabernet': 'Cabernet',
    'Cinza': 'Gris',
    'Cinza espacial': 'Gris Espacial',
    'Cobre': 'Cobre',
    'Coral': 'Coral',
    'Couro Marrom': 'Cuero Marrón',
    'Couro Navy': 'Cuero Navy',
    'Couro Vinho': 'Cuero Vino',
    'Couro Vintage': 'Cuero Vintage',
    'Cromo': 'Cromo',
    'Dourado': 'Dorado',
    'Framboesa': 'Frambuesa',
    'Indigo': 'Indigo',
    'Iuna': 'Madera',
    'Olympic Edition': 'Edicion Olimpica',
    'Ouro': 'Oro',
    'Ouro Rosa': 'Oro Rosa',
    'Platinum': 'Platino',
    'Prata': 'Plata',
    'Prateado': 'Plateado',
    'Preto': 'Negro',
    'Preto Asfalto': 'Negro Asfalto',
    'Preto Azul': 'Negro,Azul',
    'Preto Azul Navy': 'Negro,Azul Navy',
    'Preto Bambu': 'Negro,Bambu',
    'Preto Branco': 'Negro,Blanco',
    'Preto Brilhante': 'Negro Brillante',
    'Preto Cabernet': 'Negro,Cabernet',
    'Preto Matte': 'Negro Mate',
    'Preto Pink': 'Negro,Rosa',
    'Preto Tabaco': 'Negro,Tabaco',
    'Preto Verde': 'Negro,Verde',
    'Preto Vermelho': 'Negro,Rojo',
    'Rosa': 'Rosa',
#     'Rose': 'Rosa',
    'Rouge': 'Rouge',
    'Roxo': 'Purpura',
#     'Silver': 'Plata',
    'Titânio': 'Titanio',
    'Turquesa': 'Turquesa',
    'Verde': 'Verde',
    'Verde Petroleo': 'Verde Petroleo',
    'Verde Água': 'Verde Agua',
    'Vermelho': 'Rojo',
    'Ônix': 'Onice'
}

df['color'] = df['color'].cat.rename_categories(color_translations)

Como no podemos renombrar dos categorías al mismo nombre, queda cambiar los valores de 'Rose' y 'Silver'.

In [34]:
df['color'].loc[df['color'] == 'Rose'] = 'Rosa'
df['color'].loc[df['color'] == 'Silver'] = 'Plata'
df['color'].cat.remove_unused_categories(inplace=True)

<h4 id="skus">skus</h4>

Los skus aparecen en aquellos eventos relacionados a la visita de la homepege (generic listing) o cuando se el usuario visita un listado específico de una marca viendo un conjunto de productos (brand listing). 

In [35]:
appear_skus = df.loc[(df['event']=='generic listing') | (df['event']=='brand listing'),:]
appear_skus.head()

,timestamp,event,person,url,sku,model,condition,storage_gb,color,skus,search_term,staticpage,campaign_source,search_engine,channel,new_vs_returning,city,region,country,device_type,screen_resolution,operating_system_version,browser_version
1760141,2018-02-09 19:33:18,generic listing,4115546b,NaN,NaN,NaN,NaN,NaN,NaN,"10296,10198,10112,7125,10029,7154,9944,7084,14...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1760142,2018-02-09 20:05:40,generic listing,4115546b,NaN,NaN,NaN,NaN,NaN,NaN,"10296,10198,10112,7125,10029,7154,9944,7084,14...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1760143,2018-02-09 20:10:22,brand listing,4115546b,NaN,NaN,NaN,NaN,NaN,NaN,"3191,6819,2783,8135,2778,2725,12436,4477,2782,...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1760144,2018-02-09 20:15:33,generic listing,59dd7001,NaN,NaN,NaN,NaN,NaN,NaN,"10198,10114,10296,9960,9917,7125,10029,7154,99...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1760145,2018-02-09 20:18:16,brand listing,4115546b,NaN,NaN,NaN,NaN,NaN,NaN,"3191,6819,2783,8135,2778,2725,12436,4477,2782,...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Se puede observar que estos eventos, además de brindarnos la hora y el usuario que lo realiza, solamente nos da los skus que se encuentran en la página. Por lo tanto, aquellos que no nos brinden información deben ser descartados:

In [36]:
undefined_skus = appear_skus.loc[(appear_skus['skus'].isnull()),:]
undefined_skus.size

25990

In [37]:
df.drop(index=undefined_skus.index, inplace=True)

In [38]:
df.loc[((df['event']=='generic listing') | (df['event']=='brand listing'))&(df['skus'].isnull()),:].size

0

Luego de la limpieza, ya no nos quedan registros de eventos de generic listing y brand listing que no tengan información útil.

<h4 id="staticpage">staticpage</h4>

In [39]:
df['staticpage'].cat.categories

Index(['AboutUs', 'Conditions', 'CustomerService', 'FaqEcommerce',
       'PrivacyEcommerce', 'Quiosks', 'TermsAndConditionsEcommerce',
       'TermsAndConditionsReturnEcommerce', 'black_friday', 'club-trocafone',
       'galaxy-s8', 'how-to-buy', 'how-to-sell', 'trust-trocafone'],
      dtype='object')

<h4 id="campaign_source">campaign_source</h4>

In [40]:
df['campaign_source'].cat.categories

Index(['Facebook', 'FacebookAds', 'FacebookSocial', 'MARKETING SOCIAL',
       'afiliado', 'afilio', 'bing', 'blog', 'buscape', 'criteo', 'datacrush',
       'emblue', 'gizmodo', 'google', 'indexa', 'manifest', 'mercadopago',
       'onsite', 'rakuten', 'rtbhouse', 'voxus', 'yotpo', 'zanox'],
      dtype='object')

<h4 id="search_engine">search_engine</h4>

In [41]:
df['search_engine'].cat.categories

Index(['Ask', 'Bing', 'Google', 'Yahoo'], dtype='object')

<h4 id="channel">channel</h4>

In [42]:
df['channel'].cat.categories

Index(['Direct', 'Email', 'Organic', 'Paid', 'Referral', 'Social', 'Unknown'], dtype='object')

Acá notamos que para los tipos de canales por los que se originó el evento existen tanto nulos como 'Unknown'. Esto es un poco extraño dado que ambos parecen aportar la misma información, es decir, que no se sabe cuál es el canal por el que se originó el evento. Para ver porqué ocurre esto investigemos un poco más que otra información hay en los eventos de tipo 'Unknown':

In [43]:
df[df['channel'] == 'Unknown'].shape

(7, 23)

Notamos que son solo 9 registros, por lo que podemos analizarlos directamente:

In [44]:
df[df['channel'] == 'Unknown']

,timestamp,event,person,url,sku,model,condition,storage_gb,color,skus,search_term,staticpage,campaign_source,search_engine,channel,new_vs_returning,city,region,country,device_type,screen_resolution,operating_system_version,browser_version
2216088,2018-04-12 16:14:40,visited site,f12bd42e,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unknown,Returning,Unknown,Maranhao,Brazil,Computer,1280x768,Windows XP,Firefox 42
2247343,2018-05-30 11:27:11,visited site,d67688d9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unknown,Returning,Fortaleza,Ceara,Brazil,Smartphone,360x640,Android 7,Chrome Mobile 64.0
2247344,2018-05-30 14:03:02,visited site,d67688d9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unknown,Returning,Fortaleza,Ceara,Brazil,Smartphone,360x640,Android 7,Chrome Mobile 64.0
2317022,2018-04-04 18:27:21,visited site,f12bd42e,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unknown,Returning,Unknown,Maranhao,Brazil,Computer,1280x768,Windows XP,Firefox 42
2317213,2018-05-20 23:20:51,visited site,a0d795fb,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unknown,New,São Paulo,Sao Paulo,Brazil,Smartphone,360x668,Android 7.1.1,Chrome Mobile 66.0
2319241,2018-05-28 20:30:18,visited site,d67688d9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unknown,New,Fortaleza,Ceara,Brazil,Smartphone,360x640,Android 7,Chrome Mobile 62.0
2319242,2018-05-28 21:34:16,visited site,d67688d9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unknown,Returning,Unknown,Unknown,Brazil,Smartphone,360x640,Android 7,Chrome Mobile 62.0


De esta forma nos damos cuenta que frente a la totalidad de registros no es un número representativo y podemos cambiarlos por  nulos (o también los nulos por 'Unknown') sin problemas.
Como este escenario lo encontramos en varios categorias realizamos este cambio con todas ellas.

In [45]:
for categoria in atributos_categoricos:
    df[categoria].loc[df[categoria] == 'Unknown'] = np.nan
    df[categoria] = df[categoria].cat.remove_unused_categories()

<h4 id="new_vs_returning">new_vs_returning</h4>

In [46]:
df['new_vs_returning'].cat.categories

Index(['New', 'Returning'], dtype='object')

<h4 id="city">city</h4>

In [47]:
df['city'].cat.categories

Index(['Abadiania', 'Abaete', 'Abaetetuba', 'Abelardo Luz', 'Abrantes',
       'Abreu e Lima', 'Acailandia', 'Acajutiba', 'Acarau', 'Acopiara',
       ...
       'Woodbridge', 'Wrexham', 'Xambioa', 'Xanxere', 'Xavantina', 'Xaxim',
       'Xexeu', 'Xinguara', 'Yellowknife', 'Ze Doca'],
      dtype='object', length=2205)

<h4 id="region">region</h4>

In [48]:
df['region'].cat.categories

Index(['Acre', 'Alabama', 'Alagoas', 'Amapa', 'Amazonas', 'Arizona',
       'Arkansas', 'Astrakhanskaya Oblast'', 'Asuncion', 'Aveiro',
       ...
       'Tokyo', 'Tul'skaya Oblast'', 'Turin', 'Tyrol', 'Utah', 'Virginia',
       'Washington', 'Western Cape', 'Wisconsin', 'Wrexham'],
      dtype='object', length=121)

<h4 id="country">country</h4>

In [49]:
df['country'].cat.categories

Index(['Algeria', 'Angola', 'Argentina', 'Austria', 'Bangladesh', 'Belgium',
       'Bolivia', 'Brazil', 'Bulgaria', 'Burundi', 'Cameroon', 'Canada',
       'Cape Verde', 'Colombia', 'Costa Rica', 'Dominican Republic', 'France',
       'Georgia', 'Germany', 'Guadeloupe', 'Guinea-Bissau', 'India', 'Ireland',
       'Israel', 'Italy', 'Jamaica', 'Japan', 'Mexico', 'Morocco',
       'Mozambique', 'Netherlands', 'Pakistan', 'Paraguay', 'Peru',
       'Philippines', 'Portugal', 'Republic of Korea', 'Romania', 'Russia',
       'Singapore', 'Slovak Republic', 'South Africa', 'Spain', 'Sweden',
       'São Tomé and Príncipe', 'Ukraine', 'United Kingdom', 'United States',
       'Uruguay', 'Vietnam'],
      dtype='object')

<h4 id="device_type">device_type</h4>

In [50]:
df['device_type'].cat.categories

Index(['Computer', 'Smartphone', 'Tablet'], dtype='object')

<h4 id="operating_system_version">operating_system_version</h4>

In [51]:
df['operating_system_version'].cat.categories

Index(['Android ', 'Android 10.0.2', 'Android 2.3.6', 'Android 3.2',
       'Android 3.2.2', 'Android 4', 'Android 4.0.3', 'Android 4.0.4',
       'Android 4.1.1', 'Android 4.1.2',
       ...
       'iOS 8.1.3', 'iOS 8.2', 'iOS 8.3', 'iOS 8.4.1', 'iOS 9.1', 'iOS 9.2',
       'iOS 9.2.1', 'iOS 9.3.2', 'iOS 9.3.4', 'iOS 9.3.5'],
      dtype='object', length=131)

<h4 id="browser_version">browser_version</h4>

In [52]:
df['browser_version'].cat.categories

Index(['Android 2.3', 'Android 3.2', 'Android 4.0', 'Android 4.1',
       'Android 4.2', 'Android 4.3', 'Android 4.4', 'Android 5.1', 'Android 7',
       'BingPreview 1',
       ...
       'UC Browser 12', 'UC Browser 12.2', 'UC Browser 12.5', 'UC Browser 7.0',
       'Vivaldi 1.95', 'Vivaldi 1.96', 'WebKit Nightly 535.20',
       'Yandex Browser 18.1', 'Yandex Browser 18.3', 'Yandex Browser 18.4'],
      dtype='object', length=366)

<h2 id="otros-atributos">Otros atributos</h2>

<h4 id="screen_resolution">screen_resolution</h4>

Este atributo es un poco particular, dado que resulta difícil clasificarlo. Esto se debe a que la resolución de una pantalla podemos pensarla como un atributo categórico, dado que la cantidad de resoluciones es limitada o como uno numérico (el resultado del ancho por el alto). Sin embargo ambas tienen desventajas: si lo consideramos categórico perdemos, en parte, la manera de realizar consultas del tipo 'resulciones más chicas que' de manera sencilla y si consideramos el ancho por el alto perdemos información sobre las dimensiones de los dispositivos.
Es por eso que se decidió dividir este atributo en dos: screen_resolution_with (ancho) y screen_resolution_height (alto) siendo ambos numéricos.

In [53]:
resolutions = df['screen_resolution'].str.split('x', expand=True)

In [54]:
df['screen_resolution_width'] = pd.to_numeric(resolutions[0])
df['screen_resolution_height'] =pd.to_numeric(resolutions[1])
df.drop(columns=['screen_resolution']);

## Devuelvo el dataframe
Para reutilizar el dataframe limpio en otras notebooks creo una función que lo devuelva.

In [55]:
df_random_name_d76fdv12i9 = df
def get_clean_df():
    return df_random_name_d76fdv12i9

Para utilizar la limpieza en otras notebooks debe ejecutarse el siguiente código:
```
%run limpieza.ipynb
df = get_clean_df()
```

### timestamp

Parseo las fechas.

In [56]:
df['timestamp'] = pd.to_datetime(df['timestamp'])

## Dataframe con features

Devuelvo un dataframe que se va a llenar con los features. Para ello creo el nuevo dataframe que va a tener una fila por usuario:

In [57]:
df_features = pd.DataFrame(index=df['person'].unique())
df_features.index.name = 'person'

Y lo devuelvo a través de una función:

In [58]:
df_features_random_name_d76fdv12i9 = df_features
def get_df_features():
    return df_features_random_name_d76fdv12i9